In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [1]:
rdd = sc.textFile('2015_07_22_mktplace_shop_web_log_sample.log')
rdd = rdd.map(lambda x: x.split(" "))

In [2]:
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *



In [3]:
import dateutil.parser
rdd = rdd.map(lambda x: ((x[2].split(':')[0]),Row(timestamp=dateutil.parser.parse(x[0]),url=x[12]))).sortBy(lambda x:(x[0],x[1].timestamp))
rdd.take(10)

[('1.186.101.79',
  Row(timestamp=datetime.datetime(2015, 7, 22, 10, 45, 55, 881199, tzinfo=tzutc()), url='https://paytm.com:443/shop/wallet/balance?channel=web&version=2')),
 ('1.186.101.79',
  Row(timestamp=datetime.datetime(2015, 7, 22, 10, 45, 55, 885488, tzinfo=tzutc()), url='https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2')),
 ('1.186.101.79',
  Row(timestamp=datetime.datetime(2015, 7, 22, 10, 46, 27, 839734, tzinfo=tzutc()), url='https://paytm.com:443/shop/v1/frequentorders?channel=web&version=2')),
 ('1.186.101.79',
  Row(timestamp=datetime.datetime(2015, 7, 22, 10, 46, 56, 591943, tzinfo=tzutc()), url='https://paytm.com:443/papi/v1/expresscart/verify')),
 ('1.186.101.79',
  Row(timestamp=datetime.datetime(2015, 7, 22, 10, 47, 1, 782695, tzinfo=tzutc()), url='https://paytm.com:443/api/v1/expresscart/checkout?wallet=1')),
 ('1.186.101.79',
  Row(timestamp=datetime.datetime(2015, 7, 22, 10, 47, 6, 893987, tzinfo=tzutc()), url='https:/

In [4]:
grouped = rdd.groupBy(lambda x: x[0]).map(lambda x:list(x[1]))

In [5]:
grouped.take(10)

[[('1.186.108.29',
   Row(timestamp=datetime.datetime(2015, 7, 22, 10, 38, 51, 826976, tzinfo=tzutc()), url='https://paytm.com:443/shop/p/asmi-collection-pvc-octopus-fish-in-sea-wall-stickers-CMPLXHOMASMI-COLLECTBOGA28128F109D763')),
  ('1.186.108.29',
   Row(timestamp=datetime.datetime(2015, 7, 22, 10, 38, 54, 755955, tzinfo=tzutc()), url='https://paytm.com:443/shop/cart')),
  ('1.186.108.29',
   Row(timestamp=datetime.datetime(2015, 7, 22, 10, 38, 56, 476670, tzinfo=tzutc()), url='https://paytm.com:443/papi/v1/promosearch/product/8138792/offers?parent_id=8138808'))],
 [('1.186.114.45',
   Row(timestamp=datetime.datetime(2015, 7, 22, 18, 2, 59, 873175, tzinfo=tzutc()), url='https://paytm.com:443/')),
  ('1.186.114.45',
   Row(timestamp=datetime.datetime(2015, 7, 22, 18, 3, 5, 453198, tzinfo=tzutc()), url='https://paytm.com:443/shop/v1/frequentorders?channel=web&version=2')),
  ('1.186.114.45',
   Row(timestamp=datetime.datetime(2015, 7, 22, 18, 3, 5, 644265, tzinfo=tzutc()), url='http

In [10]:
class Session:
    def __init__(self, IP, sessionNumUrls,sessionLengthInMinutes):
        self.IP = IP
        self.sessionNumUrls = sessionNumUrls
        self.sessionLengthInMinutes = sessionLengthInMinutes

In [49]:
import datetime

def Sessionize(x):
    size = len(x)
    sessionUrls = [set([x[0][1].url])]
    sessions = [[x[0][1].timestamp]]
    sessionLengthInMinutes = []
    sessionNumUrls = []
    
    for i in range(1,size,1):
        if (x[i][1].timestamp - sessions[-1][-1]).total_seconds()/60 <= 15:
            sessions[-1].append(x[i][1].timestamp)
            sessionUrls[-1].add(x[i][1].url)
        else:
            sessions.append([x[i][1].timestamp]) #start a new session
            sessionUrls.append(set([x[i][1].url]))

    for session in sessions:
        sessionLengthInMinutes.append((session[-1]-session[0]).total_seconds()/60)
    
    for sessionUrl in sessionUrls:
        sessionNumUrls.append(len(sessionUrl))
    
    return Session(x[0][0], sessionNumUrls, sessionLengthInMinutes)

In [50]:
grouped_sessionized = grouped.map(lambda x:Sessionize(x))

In [52]:
grouped_sessionized.map(lambda x:(x.IP, x.sessionNumUrls, x.sessionLengthInMinutes)).sortBy(lambda x:len(x[2]),ascending=False).take(5)

[('220.226.206.7',
  [7, 5, 25, 13, 42, 15, 61, 105, 50, 92, 284, 30, 163],
  [3.0329445333333336,
   3.0448327333333336,
   4.056144683333334,
   4.024646466666667,
   34.0800585,
   4.8486105833333335,
   4.8596510833333335,
   4.173275283333333,
   13.028803033333332,
   4.2956905,
   25.08101915,
   4.992088083333333,
   3.7483676833333335]),
 ('119.81.61.166',
  [3334, 1100, 1671, 489, 1739, 8016, 2786, 3928, 3637, 2841],
  [4.950189583333333,
   4.9740852166666665,
   4.986045,
   4.2217356,
   34.4808075,
   14.613619533333333,
   4.337334383333333,
   4.9999253333333336,
   4.986084966666667,
   4.993532816666667]),
 ('54.250.253.236',
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [4.07740025,
   1.5254125,
   4.57557975,
   3.0549327,
   33.58166476666666,
   12.720924966666667,
   4.073001533333334,
   3.0533205,
   3.5601389833333332,
   2.5412049666666667]),
 ('54.251.151.39',
  [3, 3, 3, 4, 7, 3, 3, 4, 3, 2],
  [4.946743883333334,
   4.976086116666666,
   4.996719733333333,
   4.213

In [53]:
import numpy as np
grouped_sessionized.map(lambda x:(x.IP,np.mean(x.sessionLengthInMinutes))).sortBy(lambda x:x[1], ascending=False).take(5)

[('103.29.159.138', 34.42355186666667),
 ('125.16.218.194', 34.407923399999994),
 ('14.99.226.79', 34.381827933333334),
 ('122.169.141.4', 34.3359822),
 ('14.139.220.98', 34.305317716666664)]